In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/D2NN.git

import sys
sys.path.append('/content/D2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from module.one_dim.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [25]:
class Modulation(tf.keras.layers.Layer):
    def __init__(self, limitation=None, phi_max=0.0):
        super(Modulation, self).__init__()
        if limitation is not None:
            self.limitation = tf.Variable(limitation, validate_shape=False, name="limitation", trainable=False)
            self.limitation = limitation
        else:
            self.limitation = tf.Variable("None", validate_shape=False, name="limitation", trainable=False)
            self.limitation = limitation

        self.phi_max = tf.Variable(phi_max, validate_shape=False, name="phi_max", trainable=False)
        assert self.phi_max.numpy() >= 0.0

    def build(self, input_dim):
        self.input_dim = input_dim
        self.phi = self.add_weight("phi",
                                   shape=[int(input_dim[-1])])
        super(Modulation, self).build(input_dim)

    @tf.function
    def get_limited_phi(self):
        if self.limitation == 'sigmoid':
            return self.phi_max * tf.sigmoid(self.phi)
        else:
            return self.phi

    def get_config(self):
        config = super().get_config()
        config.update({
            "limitation": self.limitation,
            "phi_max": self.phi_max.numpy()
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    @tf.function
    def call(self, x):
        phi = self.get_limited_phi()
        return x * tf.complex(tf.cos(phi), tf.sin(phi))

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
#@title デフォルトのタイトル テキスト
wavelength = 800.0e-9 #@param {type:"number"}
d = 0.5e-6 #@param {type:"number"}
n = 1.0 #@param {type:"number"}

In [23]:
def create_model(**kwargs):
    inputs = tf.keras.Input((28, 28))
    x = ImageResizing([14, 14])(inputs)
    x = ImageTo1D()(x)
    x = IntensityToElectricField()(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=50.0e-6, d=d, n=n, method="expand")(x)
    x = Modulation()(x)
    x = AngularSpectrum(wavelength=wavelength, z=50.0e-6, d=d, n=n, method="expand")(x)
    x = ElectricFieldToIntensity()(x)
    x = Detector(10, padding=0.2)(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
model = create_model()
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
              metrics=['accuracy'])

epochs = 30
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0.05,
    patience=2,
)

model_name = "20220408_" + str(i + 1)
cholab_path = "/content/drive/MyDrive/D2NN/"
checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

result = model.fit(x_train,
                   y_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_test, y_test),
                   callbacks=[cp_callback]
                   )

path = cholab_path + "trained_model/"+ model_name
model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")

Metal device set to: Apple M1


2022-06-04 08:12:56.985195: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-04 08:12:56.985309: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing (ImageResizi  (None, 14, 14)           0         
 ng)                                                             
                                                                 
 image_to1d (ImageTo1D)      (None, 196)               0         
                                                                 
 intensity_to_electric_field  (None, 196)              0         
  (IntensityToElectricField)                                     
                                                                 
 modulation (Modulation)     (None, 196)               197       
                                                                 
 angular_spectrum (AngularSp  (None, 196)              5     

2022-06-04 08:12:57.508098: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-04 08:12:57.690449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - ETA: 0s - loss: 2.1287 - accuracy: 0.2075

2022-06-04 08:13:37.321509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 43s 45ms/step - loss: 2.1287 - accuracy: 0.2075 - val_loss: 2.0070 - val_accuracy: 0.2173
Epoch 2/30
938/938 [==============================] - 42s 45ms/step - loss: 1.8950 - accuracy: 0.2750 - val_loss: 1.7786 - val_accuracy: 0.3740
Epoch 3/30
938/938 [==============================] - 42s 45ms/step - loss: 1.7118 - accuracy: 0.4782 - val_loss: 1.6386 - val_accuracy: 0.5851
Epoch 4/30
938/938 [==============================] - 43s 46ms/step - loss: 1.5999 - accuracy: 0.6206 - val_loss: 1.5415 - val_accuracy: 0.6804
Epoch 5/30
938/938 [==============================] - 42s 45ms/step - loss: 1.5296 - accuracy: 0.6852 - val_loss: 1.4927 - val_accuracy: 0.7137
Epoch 6/30
938/938 [==============================] - 43s 46ms/step - loss: 1.4965 - accuracy: 0.7055 - val_loss: 1.4681 - val_accuracy: 0.7205
Epoch 7/30
938/938 [==============================] - 42s 45ms/step - loss: 1.4768 - accuracy: 0.7140 - val_loss: 1.4506 - val_accuracy: 0.7328
Epo

In [26]:
model = create_model()
model.save("trained_model/test")

INFO:tensorflow:Assets written to: trained_model/test/assets


In [9]:
fig = plt.figure(figsize=(6, 6))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
ax1.plot(range(1, epochs + 1), result.history['loss'], color='black', label='Loss')
ax1.set_ylabel('Loss')
ax1.set_xlabel('epoch')
ax2.plot(range(1, epochs + 1), result.history['accuracy'], color='red', label='Accuracy')
ax2.set_ylabel('Accuracy')
# label1と2には、凡例用に各labelのリスト情報が入る
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
# 凡例をまとめて出力する
ax1.legend(handler1 + handler2, label1 + label2, loc=2, borderaxespad=0.)

In [11]:
model.evaluate(x_test, y_test)

2022-06-04 08:37:49.790385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
